# Data cleaning plans
The goal of this notebook is to take the CSV of addresses where we clean hot tubs, geo code those adresses and create a matrix which with the distances between all the hot tubs to begin working on clustering
### Steps
1. import data 
2. use geopy and geopandas to locate each point (verrifying cords along the way) and add the geo code to the data and re write to csv
3. Usevthose geocodes and the open street map to create a distance matrix to use to plan routes. Save this file to CSV


## Step 1 importing the data 

In [1]:
# import neeed librarys
import geopandas
import geopy
import pandas as pd
import numpy as np
import folium
import requests
import polyline
import json
from geopy.geocoders import Nominatim
locator = Nominatim(user_agent="Hottub_Route_Data_Cleaning")

In [2]:
# import data from Address CSV
weekly = pd.read_excel ('WeeklyMasterList.xlsx')
weekly
weekly=weekly.set_index('ID')
nweekly=pd.read_csv('n.csv') 

## Step 2: Get geocodes for Each tub

In [3]:
# 1 - conveneint function to delay between geocoding calls
geocode = locator.geocode
# Call the API to get Geocodes for all
weekly['ADDRESS1']=weekly['ADDRESS']+', '+weekly['State']
weekly['location'] = weekly['ADDRESS1'].apply(geocode)
#print(df['location'])
# 3 - create longitude, laatitude and altitude from location column (returns tuple)
weekly['point'] = weekly['location'].apply(lambda loc: tuple(loc.point[0:2]) if loc else None)
#print(df['point'])
# 4 - split point column into latitude, longitude and altitude columns
weekly[['latitude', 'longitude']] = pd.DataFrame(weekly['point'].tolist(), index=weekly.index)
weekly

,State,ADDRESS,Sheet,Week,Order,ADDRESS1,location,point,latitude,longitude
ID,,,,,,,,,,
1AB1,"Garrett County, Maryland",1540 PENN POINT,1,AB,1,"1540 PENN POINT, Garrett County, Maryland","(1540, Penn Point Road, Garrett County, Maryla...","(39.4679859667983, -79.30870737735493)",39.467986,-79.308707
1AB2,"Garrett County, Maryland",495 PENN COVE ROAD,1,AB,2,"495 PENN COVE ROAD, Garrett County, Maryland","(495, Penn Cove Road, Garrett County, Maryland...","(39.46733899012339, -79.31634545930561)",39.467339,-79.316345
1AB3,"Garrett County, Maryland",445 PENN COVE ROAD,1,AB,3,"445 PENN COVE ROAD, Garrett County, Maryland","(445, Penn Cove Road, Garrett County, Maryland...","(39.46734561148642, -79.31733044256482)",39.467346,-79.317330
1A4,"Garrett County, Maryland",2407 BOY SCOUT ROAD,1,A,4,"2407 BOY SCOUT ROAD, Garrett County, Maryland","(2407, Boy Scout Road, Garrett County, Marylan...","(39.48698366848716, -79.30222436746641)",39.486984,-79.302224
1AB5,"Garrett County, Maryland",137 EAGLE SCOUT LANE,1,AB,5,"137 EAGLE SCOUT LANE, Garrett County, Maryland","(137, Eagle Scout Lane, Garrett County, Maryla...","(39.488699993275944, -79.30505505121599)",39.488700,-79.305055
...,...,...,...,...,...,...,...,...,...,...
12AB14,"Garrett County, Maryland",88 LAKE FOREST DRIVE,12,AB,14,"88 LAKE FOREST DRIVE, Garrett County, Maryland","(88, Lake Forest Drive, McComas Beach, Garrett...","(39.50316452005349, -79.3783936169262)",39.503165,-79.378394
12AB15,"Garrett County, Maryland",30 LAKE FOREST DRIVE,12,AB,15,"30 LAKE FOREST DRIVE, Garrett County, Maryland","(30, Lake Forest Drive, McComas Beach, Garrett...","(39.50258659934219, -79.37768709495147)",39.502587,-79.377687
12AB16,"Garrett County, Maryland",246 McComas Beach Terrace,12,AB,16,"246 McComas Beach Terrace, Garrett County, Ma...","(246, McComas Beach Terrace, McComas Beach, Ga...","(39.50194327878323, -79.37116675849465)",39.501943,-79.371167


In [4]:
# create refrences to each current route
routes = weekly.groupby("Sheet")
r1=weekly[weekly["Sheet"]==1]
r2=weekly[weekly["Sheet"]==2]
r3=weekly[weekly["Sheet"]==3]
r4=weekly[weekly["Sheet"]==4]
r5=weekly[weekly["Sheet"]==5]
r6=weekly[weekly["Sheet"]==6]
r7=weekly[weekly["Sheet"]==7]
r8=weekly[weekly["Sheet"]==8]
r9=weekly[weekly["Sheet"]==9]
r11=weekly[weekly["Sheet"]==11]
r12=weekly[weekly["Sheet"]==12]

In [12]:
# Verrify the geocodes by sheet
#NA=weekly.iloc(:,weekly.)
#NA=weekly.query("point=='None'")

#r1
#r2
#r3
#r4
r5
#r6
#r7
#r8
#r9
#r11
#r12

,State,ADDRESS,Sheet,Week,Order,ADDRESS1,location,point,latitude,longitude
ID,,,,,,,,,,
5AB1,"Garrett County, Maryland",34 HIDDEN VALLEY LANE,5,AB,1,"34 HIDDEN VALLEY LANE, Garrett County, Maryland","(34, Hidden Valley Lane, Garrett County, Maryl...","(39.49261079840201, -79.2625904939502)",39.492611,-79.262590
5AB2,"Garrett County, Maryland",336 WOODLAND WAY,5,AB,2,"336 WOODLAND WAY , Garrett County, Maryland","(336, Woodland Way, Garrett County, Maryland, ...","(39.493719933354015, -79.26779883683747)",39.493720,-79.267799
5AB3,"Garrett County, Maryland",115 WOODLAND WAY,5,AB,3,"115 WOODLAND WAY , Garrett County, Maryland","(115, Woodland Way, Garrett County, Maryland, ...","(39.49333786153111, -79.26319448332703)",39.493338,-79.263194
5AB4,"Garrett County, Maryland",41 RIDGE ROAD,5,AB,4,"41 RIDGE ROAD, Garrett County, Maryland","(41, Ridge Road, Garrett County, Maryland, 215...","(39.49301856855102, -79.26579088130933)",39.493019,-79.265791
5AB5,"Garrett County, Maryland",42 HIGH POINT DRIVE,5,AB,5,"42 HIGH POINT DRIVE , Garrett County, Maryland","(42, High Point Drive, Garrett County, Marylan...","(39.49598834544649, -79.26461608301953)",39.495988,-79.264616
5AB6,"Garrett County, Maryland",1041 SKY VALLEY ROAD,5,AB,6,"1041 SKY VALLEY ROAD , Garrett County, Maryland","(Sky Valley Road, Garrett County, Maryland, 21...","(39.4901822, -79.2541047)",39.490182,-79.254105
5AB7,"Garrett County, Maryland",655 SKY VALLEY DRIVE,5,AB,7,"655 SKY VALLEY DRIVE, Garrett County, Maryland","(655, Sky Valley Drive, Garrett County, Maryla...","(39.497875413781685, -79.26266841474519)",39.497875,-79.262668
5AB8,"Garrett County, Maryland",610 TRACI LANE,5,AB,8,"610 TRACI LANE, Garrett County, Maryland","(610, Traci Lane, Garrett County, Maryland, 21...","(39.50427328737966, -79.2573140649762)",39.504273,-79.257314
5AB9,"Garrett County, Maryland",11 VALLEY VIEW,5,AB,9,"11 VALLEY VIEW, Garrett County, Maryland","(11, Valley View Drive, Garrett County, Maryla...","(39.49513291526877, -79.26085466770733)",39.495133,-79.260855


In [6]:
location = locator.reverse((39.48140661169957, -79.26196332684248))
print(location)
location = locator.geocode("245 shore point rd Maryland")
print(location)

Garrett County, Maryland, 21561, United States
None


In [7]:
# Add routes to the CSV
weekly.to_csv("weeklys.csv")


## Step 3: Distance matrix

In [8]:
## function for getting the distance between 2 points from the api

In [9]:
## Code to get routes from the geo codes

In [10]:
## verifying the routes 

In [11]:
#write the routes to a sheet